# Pandas - Vlastní kalendáře

## Co jsou vlastní kalendáře a kdy je použít?

Vlastní kalendáře (custom calendars) umožňují definovat pracovní dny podle specifických potřeb. V meteorologii a analýze dat mohou být užitečné pro:

- **Plánování měření** - některá měření se neprovádějí o svátcích
- **Analýzu vlivů** - vliv pracovních dnů na kvalitu ovzduší
- **Reportování** - výpočet statistik pouze pro pracovní dny
- **Mezinárodní projekty** - různé země mají různé svátky

### Základní koncepty:

1. **Holiday** - definice svátku (jednorázový nebo opakující se)
2. **AbstractHolidayCalendar** - základní třída pro vytvoření kalendáře
3. **CustomBusinessDay** - offset respektující vlastní kalendář

## Přehled tříd a metod

### Holiday třídy

| Třída | Popis | Příklad použití |
|-------|--------|-----------------|
| `Holiday` | Definice svátku | `Holiday('Nový rok', month=1, day=1)` |
| `AbstractHolidayCalendar` | Základní třída kalendáře | Dědit pro vlastní kalendář |
| `HolidayCalendarFactory` | Továrna na kalendáře | Vytvoření kalendáře dynamicky |

### Parametry Holiday

| Parametr | Popis | Příklad |
|----------|-------|---------|
| `name` | Název svátku | `'Vánoce'` |
| `month` | Měsíc | `12` |
| `day` | Den | `25` |
| `offset` | Posun data | `[pd.Easter(), pd.DateOffset(days=-2)]` |
| `observance` | Pravidlo náhrady | `nearest_workday` |
| `start_date` | Platnost od | `pd.Timestamp('2020-01-01')` |
| `end_date` | Platnost do | `pd.Timestamp('2030-12-31')` |

### Easter a pohyblivé svátky

| Funkce/Metoda | Popis | Příklad použití |
|---------------|-------|-----------------|
| `pd.Easter()` | Výpočet data Velikonoční neděle | `pd.Easter()` |
| `pd.DateOffset(days=n)` | Posun o n dní | `pd.DateOffset(days=-2)` pro Velký pátek |
| Kombinace | Definice pohyblivého svátku | `offset=[pd.Easter(), pd.DateOffset(days=1)]` |

### Observance funkce

| Funkce | Popis | Použití |
|--------|--------|---------|
| `nearest_workday` | Nejbližší pracovní den | Když svátek padne na víkend |
| `next_monday` | Následující pondělí | Přesun na pondělí |
| `previous_friday` | Předchozí pátek | Přesun na pátek |

### CustomBusinessDay parametry

| Parametr | Popis | Příklad použití |
|----------|--------|-----------------|
| `calendar` | Použitý kalendář | `calendar=CzechCalendar()` |
| `weekmask` | Pracovní dny v týdnu | `'Mon Tue Wed Thu Fri'` nebo `'Mon Tue Wed Thu Fri Sat'` |
| `holidays` | Seznam svátků | `holidays=['2023-01-01', '2023-12-25']` |
| `n` | Počet pracovních dnů | `CustomBusinessDay(n=2)` pro 2 pracovní dny |

### Weekmask formáty

| Formát | Popis | Příklad |
|--------|-------|---------|
| Zkratky dnů | 3-písmenné zkratky dnů | `'Mon Tue Wed Thu Fri'` |
| Binární řetězec | 1=pracovní, 0=volno | `'1111100'` (Po-Pá pracovní) |
| Rozšířený týden | Včetně soboty | `'Mon Tue Wed Thu Fri Sat'` |

### Metody pro práci s daty

| Metoda/Atribut | Popis | Příklad použití |
|----------------|-------|-----------------|
| `.dayofweek` | Číslo dne v týdnu (0=Po, 6=Ne) | `datum.dayofweek < 5` pro Po-Pá |
| `.difference()` | Odečtení DatetimeIndexu | `pracovni.difference(svatky)` |
| `.isin()` | Test přítomnosti v seznamu | `~dny.isin(svatky)` |
| `.weekday` | Číslo dne v týdnu (0=Po, 6=Ne) | `datum.weekday == 0` pro pondělí |

### Metody kalendáře

| Metoda | Popis | Příklad použití |
|--------|--------|-----------------|
| `.holidays()` | Seznam svátků v období | `cal.holidays(start, end)` |
| `.merge()` | Sloučení kalendářů | `cal1.merge(cal2, inplace=True)` |
| `.rules` | Seznam pravidel svátků | `cal.rules` |

## Vytvoření jednoduchého kalendáře

### České státní svátky

In [ ]:
import pandas as pd
import numpy as np
from pandas.tseries.holiday import Holiday, AbstractHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import datetime as dt

# ----- DEFINICE ČESKÝCH STÁTNÍCH SVÁTKŮ -----
# AbstractHolidayCalendar je základní třída pro vytváření vlastních kalendářů svátků
# Dědíme z ní a definujeme vlastní pravidla pro české svátky
class CzechCalendar(AbstractHolidayCalendar):
    """Kalendář českých státních svátků"""
    
    # rules obsahuje seznam všech svátků a jejich pravidel
    rules = [
        # Pevné svátky - mají stejné datum každý rok
        Holiday('Nový rok', month=1, day=1),
        
        # Pohyblivé svátky - datum se mění podle Velikonoc
        # pd.Easter() vypočítá datum Velikonoční neděle
        # offset přidává/odebírá dny od tohoto data
        Holiday('Velký pátek', month=1, day=1, offset=[pd.Easter(), pd.DateOffset(days=-2)]),
        # Velký pátek je 2 dny před Velikonoční nedělí
        
        Holiday('Velikonoční pondělí', month=1, day=1, offset=[pd.Easter(), pd.DateOffset(days=1)]),
        # Velikonoční pondělí je 1 den po Velikonoční neděli
        
        # Další pevné svátky
        Holiday('Svátek práce', month=5, day=1),
        Holiday('Den vítězství', month=5, day=8),
        Holiday('Den slovanských věrozvěstů', month=7, day=5),
        Holiday('Den upálení mistra Jana Husa', month=7, day=6),
        Holiday('Den české státnosti', month=9, day=28),
        Holiday('Den vzniku samostatného československého státu', month=10, day=28),
        Holiday('Den boje za svobodu a demokracii', month=11, day=17),
        Holiday('Štědrý den', month=12, day=24),
        Holiday('1. svátek vánoční', month=12, day=25),
        Holiday('2. svátek vánoční', month=12, day=26),
    ]

# ----- VYTVOŘENÍ INSTANCE KALENDÁŘE -----
# Vytvoříme instanci našeho kalendáře pro další použití
czech_cal = CzechCalendar()

# ----- ZÍSKÁNÍ SEZNAMU SVÁTKŮ PRO KONKRÉTNÍ ROK -----
# holidays() vrací DatetimeIndex se všemi svátky v zadaném období
# start a end definují rozsah, pro který chceme svátky získat
svatky_2023 = czech_cal.holidays(start='2023-01-01', end='2023-12-31')
# Výsledek: DatetimeIndex obsahující všechny české svátky roku 2023

print("České státní svátky v roce 2023:")
for svatek in svatky_2023:
    # Převedeme datum na Timestamp pro formátování
    # strftime('%A') vrací název dne v týdnu (Monday, Tuesday...)
    den_v_tydnu = pd.Timestamp(svatek).strftime('%A')
    # Vypíšeme datum v českém formátu DD.MM.YYYY
    print(f"{svatek.strftime('%d.%m.%Y')} ({den_v_tydnu})")

# Spočítáme celkový počet svátků
print(f"\nCelkem svátků: {len(svatky_2023)}")
# V ČR je 13 státních svátků + Velikonoce se mohou posunout do jiného roku

### Použití CustomBusinessDay

In [ ]:
# ----- VYTVOŘENÍ CUSTOM BUSINESS DAY S ČESKÝMI SVÁTKY -----
# CustomBusinessDay je speciální offset, který počítá pouze pracovní dny
# Parametr calendar umožňuje přidat vlastní kalendář svátků
czech_bd = CustomBusinessDay(calendar=czech_cal)
# Výsledek: Offset, který při počítání přeskakuje víkendy A české státní svátky

# ----- DEMONSTRACE ROZDÍLU MEZI BĚŽNÝM A ČESKÝM BUSINESS DAY -----
# Vybereme pátek před Velikonocemi 2023 pro názornou ukázku
start_date = pd.Timestamp('2023-04-07')  # pátek před Velikonocemi
# 7.4.2023 je pátek, 10.4.2023 je Velikonoční pondělí (státní svátek)

print(f"Výchozí datum: {start_date.strftime('%d.%m.%Y (%A)')}")
print("\nPřičítání pracovních dnů:")

# ----- BĚŽNÝ BUSINESS DAY (BDay) -----
# pd.offsets.BDay počítá pouze pracovní dny (pondělí-pátek)
# IGNORUJE státní svátky - bere v úvahu pouze víkendy
print("\nBěžný business day (ignoruje svátky):")
for i in range(1, 6):
   # BDay(i) přičte i pracovních dnů k datu
   datum = start_date + pd.offsets.BDay(i)
   print(f"+ {i} BD: {datum.strftime('%d.%m.%Y (%A)')}")
   # Očekávaný výstup:
   # + 1 BD: 10.04.2023 (Monday) - přeskočí víkend, ALE nezná svátky
   # + 2 BD: 11.04.2023 (Tuesday)
   # atd.

# ----- ČESKÝ BUSINESS DAY (czech_bd) -----
# Náš vlastní offset, který respektuje české svátky
print("\nČeský business day (respektuje svátky):")
for i in range(1, 6):
   # czech_bd * i násobí offset číslem i (přičte i českých pracovních dnů)
   datum = start_date + czech_bd * i
   print(f"+ {i} CBD: {datum.strftime('%d.%m.%Y (%A)')}")
   # Očekávaný výstup:
   # + 1 CBD: 11.04.2023 (Tuesday) - přeskočí víkend A Velikonoční pondělí
   # + 2 CBD: 12.04.2023 (Wednesday)
   # atd.
   
# ----- VYSVĚTLENÍ ROZDÍLU -----
# Upozornění na konkrétní svátek v našem příkladu
print("\nPoznámka: 10.4. je Velikonoční pondělí (svátek)")
# Běžný BDay tento den započítá jako pracovní
# Český business day ho správně přeskočí

## Práce s pracovními dny

### Počet pracovních dnů v měsíci

In [ ]:
# ----- VÝPOČET PRACOVNÍCH DNŮ PRO KAŽDÝ MĚSÍC ROKU 2023 -----
# pd.date_range s freq='MS' vytvoří seznam prvních dnů každého měsíce
# MS = Month Start (začátek měsíce)
mesice_2023 = pd.date_range('2023-01-01', '2023-12-31', freq='MS')
# Výsledek: DatetimeIndex s daty: 2023-01-01, 2023-02-01, ..., 2023-12-01

# Seznam pro uložení statistik jednotlivých měsíců
pracovni_dny_mesice = []

# ----- PROCHÁZENÍ JEDNOTLIVÝCH MĚSÍCŮ -----
for mesic in mesice_2023:
   # ----- URČENÍ ROZSAHU MĚSÍCE -----
   # start je první den měsíce (už máme z mesice_2023)
   start = mesic
   # MonthEnd(0) vrací poslední den AKTUÁLNÍHO měsíce
   # MonthEnd(1) by vrátil poslední den NÁSLEDUJÍCÍHO měsíce
   end = mesic + pd.offsets.MonthEnd(0)
   
   # ----- ZÍSKÁNÍ VŠECH DNŮ V MĚSÍCI -----
   # freq='D' znamená denní frekvence (každý den)
   vsechny_dny = pd.date_range(start, end, freq='D')
   # Např. pro leden: 1.1., 2.1., 3.1., ..., 31.1.
   
   # ----- FILTROVÁNÍ PRACOVNÍCH DNŮ (PO-PÁ) -----
   # dayofweek vrací číslo dne v týdnu: 0=pondělí, 1=úterý, ..., 6=neděle
   # Podmínka < 5 vybere pouze pondělí až pátek (0-4)
   pracovni = vsechny_dny[vsechny_dny.dayofweek < 5]
   # Výsledek: pouze dny pondělí-pátek v daném měsíci
   
   # ----- ODEČTENÍ STÁTNÍCH SVÁTKŮ -----
   # Získáme svátky, které padají do aktuálního měsíce
   svatky_mesic = czech_cal.holidays(start, end)
   # difference() odstraní ze seznamu pracovních dnů ty, které jsou svátky
   pracovni_bez_svatku = pracovni.difference(svatky_mesic)
   # Výsledek: pracovní dny minus státní svátky = skutečné pracovní dny
   
   # ----- ULOŽENÍ STATISTIK PRO MĚSÍC -----
   pracovni_dny_mesice.append({
       'mesic': mesic.strftime('%B %Y'),          # Název měsíce a rok (např. "January 2023")
       'pocet_dni': len(vsechny_dny),            # Celkový počet dnů v měsíci (28-31)
       'pracovni_dny': len(pracovni),            # Počet po-pá bez ohledu na svátky
       'svatky': len(svatky_mesic),              # Počet svátků v měsíci
       'pracovni_bez_svatku': len(pracovni_bez_svatku)  # Skutečné pracovní dny
   })

# ----- VYTVOŘENÍ PŘEHLEDNÉ TABULKY -----
# Převedeme seznam slovníků na DataFrame pro lepší zobrazení
prehled_pd = pd.DataFrame(pracovni_dny_mesice)

print("Přehled pracovních dnů v roce 2023:")
print(prehled_pd)
# Výstup: Tabulka s 12 řádky (měsíce) a 5 sloupci (statistiky)

# ----- CELKOVÝ SOUČET PRACOVNÍCH DNŮ -----
# .sum() sečte hodnoty ve sloupci 'pracovni_bez_svatku' přes všechny měsíce
print(f"\nCelkem pracovních dnů v roce 2023: {prehled_pd['pracovni_bez_svatku'].sum()}")
# Typicky kolem 250-252 pracovních dnů v roce

### Generování dat pouze pro pracovní dny

In [ ]:
# Generování pracovních dnů s českým kalendářem
start = pd.Timestamp('2023-06-01')
end = pd.Timestamp('2023-06-30')

# Metoda 1: pomocí CustomBusinessDay
pracovni_dny_cerven = pd.date_range(start, end, freq=czech_bd)

print(f"Pracovní dny v červnu 2023: {len(pracovni_dny_cerven)}")
print("\nSeznam pracovních dnů:")
for den in pracovni_dny_cerven:
    print(f"{den.strftime('%d.%m. (%A)')}")

# Metoda 2: manuální filtrace
vsechny_dny = pd.date_range(start, end, freq='D')
svatky_cerven = czech_cal.holidays(start, end)
pracovni_manual = vsechny_dny[(vsechny_dny.dayofweek < 5) & (~vsechny_dny.isin(svatky_cerven))]

print(f"\nKontrola: {len(pracovni_manual)} pracovních dnů")
print(f"Svátky v červnu: {len(svatky_cerven)}")

### Vlastní weekmask

In [ ]:
# ----- VLASTNÍ PRACOVNÍ TÝDEN (PO-SO) -----
# Některé provozy pracují i v sobotu
# weekmask definuje pracovní dny: 'Mon Tue Wed Thu Fri Sat'
cbd_po_so = CustomBusinessDay(
    weekmask='Mon Tue Wed Thu Fri Sat',  # Pracuje se Po-So
    calendar=czech_cal                    # Respektuje české svátky
)

# ----- UKÁZKA ROZDÍLU -----
datum = pd.Timestamp('2023-06-30')  # Pátek

print("Přičtení 1 pracovního dne:")
print(f"Výchozí datum: {datum.strftime('%d.%m.%Y (%A)')}")

# Standardní pracovní den (Po-Pá)
print(f"Po-Pá: {(datum + czech_bd).strftime('%d.%m.%Y (%A)')}")
# Přeskočí sobotu a neděli → pondělí

# Rozšířený pracovní den (Po-So)  
print(f"Po-So: {(datum + cbd_po_so).strftime('%d.%m.%Y (%A)')}")
# Sobota je pracovní → sobota

# ----- POČET PRACOVNÍCH DNŮ V MĚSÍCI -----
print("\nPracovní dny v červnu 2023:")
print(f"Po-Pá: {len(pd.date_range('2023-06-01', '2023-06-30', freq=czech_bd))} dnů")
print(f"Po-So: {len(pd.date_range('2023-06-01', '2023-06-30', freq=cbd_po_so))} dnů")
# Po-So má o 4 dny více (4 pracovní soboty)